In [29]:
import numpy as np
from numpy.linalg import norm
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cvxpy as cp
import proxsuite
from matplotlib.patches import Polygon,Circle


 # 지금 여기서 일단은 11월 24일에 만든 1124_fvo_cbf_no_divide_many_case 파일에서 지금  vo와 그냥 cbf을 추가하여 비교를 위한 파일임
 

In [30]:
def create_safe_initial_conditions(N_AGENTS, V_CONST, critical_distance, position_range=(-50, 50), max_attempts=10000):
    """
    Generates a random initial state matrix that satisfies the FVO-CBF safety conditions.
    Safety Condition: B2(x_ij) = p_ij^T v_ij + ||v_ij|| * sqrt(||p_ij||^2 - d^2) >= 0

    Args:
        N_AGENTS (int): Number of agents.
        V_CONST (float): Constant speed for the fixed-speed model.
        critical_distance (float): Critical distance for collision (d).
        position_range (tuple): Random generation range for position (px, py).
        max_attempts (int): Maximum number of attempts to find a valid condition.

    Returns:
        np.ndarray: A valid 5xN_AGENTS initial state matrix.
    """
    print("Searching for safe initial conditions...")
    for attempt in range(max_attempts):
        initial_states_matrix = np.zeros((5, N_AGENTS))

        # 1. Generate random positions and headings
        px = np.random.uniform(position_range[0], position_range[1], size=N_AGENTS)
        py = np.random.uniform(position_range[0], position_range[1], size=N_AGENTS)
        theta = np.random.uniform(0, 2 * np.pi, size=N_AGENTS)

        # 2. Construct the state matrix (velocity is determined from heading)
        initial_states_matrix[0, :] = px
        initial_states_matrix[1, :] = py
        initial_states_matrix[4, :] = theta
        initial_states_matrix[2, :] = V_CONST * np.cos(theta)
        initial_states_matrix[3, :] = V_CONST * np.sin(theta)

        # 3. Verify safety conditions for all pairs of agents
        all_pairs_safe = True
        for i in range(N_AGENTS):
            for j in range(i + 1, N_AGENTS):
                p_i = initial_states_matrix[0:2, i]
                p_j = initial_states_matrix[0:2, j]
                v_i = initial_states_matrix[2:4, i]
                v_j = initial_states_matrix[2:4, j]

                p_ij = p_i - p_j
                v_ij = v_i - v_j

                pij_norm = np.linalg.norm(p_ij)
                vij_norm = np.linalg.norm(v_ij)

                # Condition 1: Initial distance must be greater than the critical distance buffer
                if pij_norm <= critical_distance * 1.5: # Using a buffer of 1.5
                    all_pairs_safe = False
                    break

                # Condition 2: B2 barrier function must be >= 0
                if pij_norm**2 - critical_distance**2 < 0:
                    all_pairs_safe = False
                    break
                
                h_b2 = np.dot(p_ij, v_ij) + vij_norm * np.sqrt(pij_norm**2 - critical_distance**2)
                if h_b2 < 10:
                    all_pairs_safe = False
                    break
            if not all_pairs_safe:
                break
        
        # 4. If all conditions are met, return the valid state matrix
        if all_pairs_safe:
            print(f"Found safe initial conditions at attempt {attempt + 1}.")
            return initial_states_matrix

    # Raise an exception if max attempts are exceeded
    raise RuntimeError(f"Could not find safe initial conditions after {max_attempts} attempts.")

In [31]:
# --- 초기 상태 설정 (5차원) ---

N_AGENTS = 5
V_CONST = 20.0
critical_distance=40
STATE_DIM = 5




In [32]:
#  # 지금 많은 랜덤 케이스를 생성하는 부분임



test_case_num=10
candidiate_initial_states = np.zeros((STATE_DIM, N_AGENTS)) # (변경) 5x10 크기
initial_test_case=np.zeros((STATE_DIM,N_AGENTS,test_case_num))


for test_case in range(test_case_num):
    candidiate_initial_states = create_safe_initial_conditions(N_AGENTS, V_CONST, critical_distance, position_range=(-500, 500), max_attempts=10000)
    initial_test_case[:,:,test_case]=candidiate_initial_states


Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 2.
Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 1.
Searching for safe initial conditions...
Found safe initial conditions at attempt 3.


In [33]:

    # -- ACS 파라미터 
beta=1/4
lamda=1
k1=0.5
k2=0.1
desired_distance=10

## 지금 해당 값이 일단 정배임으로 왠만하면은 건들지 말기 
# beta=1/4
# lamda=5
# k1=0.5
# k2=0.05
# desired_distance=50




    # CBF parameter
    
class_k=0.1

In [34]:
np.save('beta.npy',beta)
np.save('lamda.npy',lamda)
np.save('k1.npy',k1)
np.save('k2.npy',k2)
np.save('desired_distance.npy',desired_distance)
np.save('class_k.npy',class_k)


In [35]:
np.save('initial_test_case.npy',initial_test_case)  
np.save('N_AGENTS.npy',N_AGENTS)
np.save('V_CONST.npy',V_CONST)
np.save('critical_distance.npy',critical_distance)
np.save('STATE_DIM.npy',STATE_DIM)
np.save('test_case_num.npy',test_case_num)



